In [1]:
import torch
import pickle
import random
import pretty_midi
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('JSB-Chorales-dataset/jsb-chorales-16th.pkl', 'rb') as p:
    data = pickle.load(p, encoding="latin1")

print(data.keys())
train_data = data['train']

dict_keys(['test', 'train', 'valid'])


In [2]:
all_chords = set(ch for sequence in train_data for ch in sequence)

#Dictionaries for chord to index and vice-version conversion
chord_to_idx = {chord: idx for idx, chord in enumerate(sorted(all_chords))}
idx_to_chord = {idx: chord for chord, idx in chord_to_idx.items()}
vocab_size = len(chord_to_idx)

tokenized_sequences = [
    [chord_to_idx[chord] for chord in sequence]
    for sequence in train_data
]

sequence_length = 32
X = []
y = []

for seq in tokenized_sequences:
    if len(seq) <= sequence_length:
        continue
    for i in range(len(seq) - sequence_length):
        X.append(seq[i:i + sequence_length])
        y.append(seq[i + sequence_length])

#Converting to tensors
X_tensor = torch.tensor(X, dtype=torch.long)
y_tensor = torch.tensor(y, dtype=torch.long)

In [3]:
class ChoraleDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 32
dataset = ChoraleDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class LSTMGenerator(nn.Module):
    def __init__(self, vocab_size, embed_size=64, hidden_size=128, num_layers=1):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        
        #Taking output from the last time step
        out = self.fc(out[:, -1, :])  
        return out, hidden

In [4]:
model = None
model = LSTMGenerator(vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=9e-4)

n_epochs = 20

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        output, _ = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}")


Epoch 1/20, Loss: 5.4419
Epoch 2/20, Loss: 3.3209
Epoch 3/20, Loss: 2.3771
Epoch 4/20, Loss: 1.8243
Epoch 5/20, Loss: 1.4818
Epoch 6/20, Loss: 1.2526
Epoch 7/20, Loss: 1.0860
Epoch 8/20, Loss: 0.9544
Epoch 9/20, Loss: 0.8467
Epoch 10/20, Loss: 0.7592
Epoch 11/20, Loss: 0.6874
Epoch 12/20, Loss: 0.6221
Epoch 13/20, Loss: 0.5711
Epoch 14/20, Loss: 0.5238
Epoch 15/20, Loss: 0.4834
Epoch 16/20, Loss: 0.4458
Epoch 17/20, Loss: 0.4209
Epoch 18/20, Loss: 0.3892
Epoch 19/20, Loss: 0.3666
Epoch 20/20, Loss: 0.3465


In [5]:
def get_random_seed(vocab_size, sequence_length):
    return [random.randint(0, vocab_size - 1) for _ in range(sequence_length)]

def generate_sequence(seed_sequence, idx_to_chord, sequence_length=32, generate_length=64):
    """
    Generates a symbolic chord sequence using a trained LSTM model
    Input:
        seed_sequence (list of int): Initial chord indices (length >= sequence_length).
        idx_to_chord (dict): Mapping from token index to chord tuple.
        sequence_length (int): Number of tokens fed to the model at each step.
        generate_length (int): Total number of new tokens to generate.
    Returns:
        list of tuple
    """
    model.eval()
    device = next(model.parameters()).device
    generated = seed_sequence[:]
    hidden = None

    for _ in range(generate_length + sequence_length):
        # Get the last `sequence_length` tokens as input
        input_seq = generated[-sequence_length:]
        input_tensor = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)

        with torch.no_grad():
            output, hidden = model(input_tensor, hidden)
            output = output[0].cpu().numpy()
            probabilities = torch.softmax(torch.tensor(output), dim=0).numpy()
            next_token = int(np.random.choice(len(probabilities), p=probabilities))
            generated.append(next_token)

    #Removing the random seed in the start to avoid noisy output
    cleaned = generated[sequence_length:]

    #Converting to chords
    return [idx_to_chord[idx] for idx in cleaned]


#Generates random seed to start off with generation
seed = get_random_seed(vocab_size, sequence_length)

generated_chords = generate_sequence(
    seed_sequence=seed,
    idx_to_chord=idx_to_chord,
    sequence_length=sequence_length,
    generate_length=64
)

In [6]:
def chords_to_midi(chords, filename, seconds_per_chord=0.25):
    """
    Converts a sequence of chords to MIDI file. The default program of Acoustic Grand Piano is used here for instrument.
    Input:
        chords (list of tuples)
        filename (str)
        seconds_per_chord (float)
    """
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    start_time = 0
    for chord in chords:
        end_time = start_time + seconds_per_chord
        for pitch in chord:
            
            note = pretty_midi.Note(
                velocity=100, pitch=int(pitch), start=start_time, end=end_time
            )
            instrument.notes.append(note)
        start_time = end_time

    midi.instruments.append(instrument)
    midi.write(filename)
    print(f"MIDI file saved as {filename}")


# Assuming `generated_chords` is your list of chord tuples from generate_sequence()

filename = "midifiles/symbolic_unconditioned.mid"
chords_to_midi(generated_chords, filename=filename, seconds_per_chord=0.25)


MIDI file saved as generated_output2.mid
